In [206]:
# !pip install dialogflow

In [1]:
# !pip install --upgrade google-cloud-speech

In [3]:
# !pip install rasa_nlu

In [8]:
# !pip install wit

In [70]:
# !pip install --upgrade google.cloud

  Using cached https://files.pythonhosted.org/packages/ba/b1/7c54d1950e7808df06642274e677dbcedba57f75307adf2e5ad8d39e5e0e/google_cloud-0.34.0-py2.py3-none-any.whl


In [30]:
# !pip install --upgrade google-cloud-language

In [60]:
# !pip install --upgrade setuptools pip

  Found existing installation: setuptools 40.6.3
    Uninstalling setuptools-40.6.3:
      Successfully uninstalled setuptools-40.6.3
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\risha\\AppData\\Local\\Temp\\pip-uninstall-f88009si\\users\\risha\\anaconda3\\scripts\\pip.exe'
Consider using the `--user` option or check the permissions.



# Libraries

In [211]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.utils import shuffle
import sklearn.linear_model as Lm
from sklearn.linear_model import LinearRegression as Lr
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
import requests
import io
import os
from google.cloud import speech
from google.cloud.speech import enums as enums1
from google.cloud.speech import types
from rasa_nlu.model import Metadata, Interpreter
from wit import Wit
import json
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from nltk import tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import json
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import dialogflow

# Functions

In [229]:
def detect_intent(text, session_id = "a"):
    language_code = 'en'
    project_id = 'Speech_To_Text'
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "NLP - Food-587336d8f0f0.json"
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)
    text_input = dialogflow.types.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.types.QueryInput(text=text_input)
    response = session_client.detect_intent(session=session, query_input=query_input)
    print('Query text: {}'.format(response.query_result.query_text))
    print('Detected intent: {} (confidence: {})\n'.format(
        response.query_result.intent.display_name,
        response.query_result.intent_detection_confidence))
    print('Fulfillment text: {}\n'.format(
        response.query_result.fulfillment_text))
    return response.query_result.fulfillment_text

In [4]:
def getTexts(Files):
    a = []
    for i in Files:
        a.append(getTextFromAudio(i))
    return a

In [3]:
def getTextFromAudio(filename):
    client = speech.SpeechClient()

    file_name = os.path.join('Audio Files',filename)

    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        enable_automatic_punctuation = True,
        encoding=enums1.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code='en-US')

    response = client.recognize(config, audio)

    for result in response.results:
        return result.alternatives[0].transcript

In [4]:
def analyze_entities(text_content):
    client = language_v1.LanguageServiceClient()
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    encoding_type = enums.EncodingType.UTF8
    response = client.analyze_entities(document, encoding_type=encoding_type)
    words = text_content.split(" ")
    d = {}
    for i in words:
        d[i] = -1
        
    for entity in response.entities:
        for mention in entity.mentions:
            if (enums.Entity.Type(entity.type).name != "NUMBER"):
                d[entity.name] = enums.EntityMention.Type(mention.type).name
            else:
                d[entity.name] = entity.metadata['value']
    return d


In [175]:
class MakeMenu:
    
    def __init__(self, database):
        self.menu = {}
        self.WitClient = Wit("QIPH5735KKLG3SRKLW6ULLPAA3FCYYIS")
        self.database = database
        self.porter = PorterStemmer()
    
    def generateMenu(self, sentences):
        mainMenu = {}
        sentences = self.removePunc(sentences)
        for i in sentences:
            if (len(i) < 2):
                continue
            i = i.lower()
            intent = self.getIntent(i)
            oneMenu = self.generateOneMenu(i)
            print(mainMenu)
            if (intent == 'Add'):
                for i in oneMenu:
                    if (i in mainMenu):
                        mainMenu[i] += oneMenu[i]
                    else:
                        mainMenu[i] = oneMenu[i]
            elif (intent == 'Delete'):
                for i in oneMenu:
                    if (i in mainMenu):
                        mainMenu[i] -= oneMenu[i]
            
        receipt = []
        for i in mainMenu:
            if i not in self.database:
                continue
            temp = []
            if (mainMenu[i] > 0):
                temp.append(i)
                temp.append(self.database[i])
                temp.append(mainMenu[i])                
                temp.append(mainMenu[i]*self.database[i])
                receipt.append(temp)
        FinalMenu = pd.DataFrame(receipt)
        FinalMenu.columns = ['Item', 'Price', 'Quantity', 'Amount']
        temp = ["", "Total", FinalMenu['Quantity'].sum(), FinalMenu['Amount'].sum()]
        FinalMenu = FinalMenu.append(pd.DataFrame([temp], columns = FinalMenu.columns),ignore_index=True)
        return FinalMenu
                
        
    def generateOneMenu(self, sentence):
        oneMenu = {}
        words, d = self.analyze_entities(sentence)
        for i in range(len(words)):
            if d[words[i]] == 'COMMON':
                quant = 1
                if (i != 0 and (str(d[words[i-1]]).isdigit()) and d[words[i-1]] != -1):
                    quant = d[words[i-1]]
                oneMenu[words[i]] = quant
        print(d)
        return oneMenu
        
        
    def getIntent(self, sentence):
        resp = self.WitClient.message(sentence)   
#         print(sentence)
        if 'intent' in resp['entities']:
            return resp['entities']['intent'][0]['value']
        else:
            return "No Intent Found"
    
    def analyze_entities(self, sentence):
        client = language_v1.LanguageServiceClient()
        type_ = enums.Document.Type.PLAIN_TEXT
        language = "en"
        sentence = sentence.split(" ")
        for i in range(len(sentence)):
            sentence[i] = sentence[i].lower()
        sentence = " ".join(sentence)
        document = {"content": sentence, "type": type_, "language": language}
        encoding_type = enums.EncodingType.UTF8
        response = client.analyze_entities(document, encoding_type=encoding_type)
        words = sentence.split(" ")
        for i in range(len(words)):
            words[i] = words[i].lower()
        d = {}
        for i in words:
            d[i] = -1

        for entity in response.entities:
            for mention in entity.mentions:
                if (enums.Entity.Type(entity.type).name != "NUMBER"):
                    d[entity.name.lower()] = enums.EntityMention.Type(mention.type).name
                else:
                    d[entity.name] = int(entity.metadata['value'])
        return words, d
    
    def removePunc(self, sentences):
        for i in range(len(sentences)):
            a = ""
            for j in sentences[i]:
                if j in punctuation:
                    a += " "
                else:
                    a += j
            sentences[i] =  a
        return sentences

        

# MAIN

In [120]:
Database = {
    'coffee' : 5,
    'tea' : 5,
    'burger' : 10,
    'sandwich' : 5,
    'pizza' : 10,
    'soda' : 3,
    'fries' : 4.5,
    'coke' : 3,
    'burritos' : 10,
    'nachos' : 5,
    'quesadilla' : 6.5
}

In [243]:
senten = []
senten.append('Can i order five Burger and fries')
senten.append('I would like to order one large pizza and one soda')
senten.append('Please remove the burger from my order')
senten.append('I Would like to get a coffee too')
senten.append('hi I would like to order one cheese burger and 5 fries and 3 coke and 7 sandwich')
senten.append('Remove one coke too')

In [65]:
senten2 = []
senten2.append('Can i get two burritos and nachos on side')
senten2.append('Add 3 Quesadilla')
senten2.append('Remove 1 burritos')
senten2.append('Can i also get three coke')

In [107]:
Files = []
for i in range(6):
    Files.append("Main_"+str(i+1)+".flac")
audioText = getTexts(Files)

In [242]:
audioText

['Hi  can I order one burger and fries ',
 'I would like to order one large pizza and one soda ',
 'Please remove the burger from my order ',
 'I would like to get a coffee  too ',
 'Hi  I would like to order one cheeseburger and five fries and three Coke and 7 Sandwich ',
 'remove one cup two']

In [244]:
a = MakeMenu(Database)
menu = a.generateMenu(senten)
menu

{'can': -1, 'i': -1, 'order': -1, 'five': 5, 'burger': 'COMMON', 'and': -1, 'fries': 'COMMON'}
{}
{'i': -1, 'would': -1, 'like': -1, 'to': -1, 'order': -1, 'one': 1, 'large': -1, 'pizza': 'COMMON', 'and': -1, 'soda': 'COMMON'}
{'burger': 5, 'fries': 1}
{'please': -1, 'remove': -1, 'the': -1, 'burger': 'COMMON', 'from': -1, 'my': -1, 'order': 'COMMON'}
{'burger': 5, 'fries': 1, 'pizza': 1, 'soda': 1}
{'i': -1, 'would': -1, 'like': -1, 'to': -1, 'get': -1, 'a': -1, 'coffee': 'COMMON', 'too': -1}
{'burger': 4, 'fries': 1, 'pizza': 1, 'soda': 1}
{'hi': -1, 'i': -1, 'would': -1, 'like': -1, 'to': -1, 'order': -1, 'one': 1, 'cheese': -1, 'burger': -1, 'and': -1, '5': 5, 'fries': 'COMMON', '3': 3, 'coke': 'COMMON', '7': 7, 'sandwich': 'COMMON', 'cheese burger': 'COMMON'}
{'burger': 4, 'fries': 1, 'pizza': 1, 'soda': 1, 'coffee': 1}
{'remove': -1, 'one': 1, 'coke': 'COMMON', 'too': -1}
{'burger': 4, 'fries': 6, 'pizza': 1, 'soda': 1, 'coffee': 1, 'coke': 3, 'sandwich': 7}


,Item,Price,Quantity,Amount
0,burger,10,4,40.0
1,fries,4.5,6,27.0
2,pizza,10,1,10.0
3,soda,3,1,3.0
4,coffee,5,1,5.0
5,coke,3,2,6.0
6,sandwich,5,7,35.0
7,,Total,22,126.0


In [108]:
a = MakeMenu(Database)
menu = a.generateMenu(audioText)
menu

hi  can i order one burger and fries 
{'hi': -1, '': -1, 'can': -1, 'i': -1, 'order': -1, 'one': 1, 'burger': 'COMMON', 'and': -1, 'fries': 'COMMON'}
{}
i would like to order one large pizza and one soda 
{'i': -1, 'would': -1, 'like': -1, 'to': -1, 'order': -1, 'one': 1, 'large': -1, 'pizza': 'COMMON', 'and': -1, 'soda': 'COMMON', '': -1}
{'burger': 1, 'fries': 1}
please remove the burger from my order 
{'please': -1, 'remove': -1, 'the': -1, 'burger': 'COMMON', 'from': -1, 'my': -1, 'order': 'COMMON', '': -1}
{'burger': 1, 'fries': 1, 'pizza': 1, 'soda': 1}
i would like to get a coffee  too 
{'i': -1, 'would': -1, 'like': -1, 'to': -1, 'get': -1, 'a': -1, 'coffee': 'COMMON', '': -1, 'too': -1}
{'burger': 0, 'fries': 1, 'pizza': 1, 'soda': 1}
hi  i would like to order one cheeseburger and five fries and three coke and 7 sandwich 
{'hi': -1, '': -1, 'i': -1, 'would': -1, 'like': -1, 'to': -1, 'order': -1, 'one': 1, 'cheeseburger': 'COMMON', 'and': -1, 'five': 5, 'fries': 'COMMON', 'thr

,Item,Price,Quantity,Amount
0,fries,4.5,6,27.0
1,pizza,10,1,10.0
2,soda,3,1,3.0
3,coffee,5,1,5.0
4,coke,3,3,9.0
5,sandwich,5,7,35.0
6,,Total,19,89.0


In [241]:
b = MakeMenu(Database)
menu = b.generateMenu(senten2)
menu

{'can': -1, 'i': -1, 'get': -1, 'two': 2, 'burritos': 'COMMON', 'and': -1, 'nachos': 'COMMON', 'on': -1, 'side': 'COMMON'}
{}
{'add': -1, '3': 3, 'quesadilla': 'COMMON'}
{'burritos': 2, 'nachos': 1, 'side': 1}
{'remove': -1, '1': 1, 'burritos': 'COMMON'}
{'burritos': 2, 'nachos': 1, 'side': 1, 'quesadilla': 3}
{'can': -1, 'i': -1, 'also': -1, 'get': -1, 'three': 3, 'coke': 'COMMON'}
{'burritos': 1, 'nachos': 1, 'side': 1, 'quesadilla': 3}


,Item,Price,Quantity,Amount
0,burritos,10,1,10.0
1,nachos,5,1,5.0
2,quesadilla,6.5,3,19.5
3,coke,3,3,9.0
4,,Total,8,43.5


# Reading Json

In [29]:
with open('Taskmaster/TM-1-2019/self-dialogs.json') as json_file:
    data = json.load(json_file)

In [35]:
pizza = []
coffee = []
for i in range(len(data)):
    if 'coffee' in data[i]['instruction_id']:
        coffee.append(i)
    if 'pizza' in data[i]['instruction_id']:
        pizza.append(i)

In [53]:
coffeel = []
for i in coffee:
    for j in data[i]['utterances']:
        coffeel.append(j['text'])
pizzal = []
for i in pizza:
    for j in data[i]['utterances']:
        pizzal.append(j['text'])

In [59]:
pd.DataFrame(coffeel).to_excel("coffee.xlsx")
pd.DataFrame(pizzal).to_excel("coffee.xlsx")

In [200]:
with open('Label/expressions-1.json') as json_file:
    main = json.load(json_file)

In [205]:
for i in main['data']:
    print(i['text'], i['entities'][0]['value'])

Please order me a burger and some fries "Add"
Modify my order please "Update"
Remove the burger from my order "Delete"
Change my burger for a pizza "Update"
I want to order 5 cheese burgers and 1 pizza "Add"
remove 5 burgers "Delete"
add 4 burgers and remove my pizza from the order "Delete"
Hi, can i order 5 fries and a coke please "Add"
Hello, i would like to order 5 pizzas  "Add"
please remove the 2 pizza from my previous order "Delete"
hello, can you get me a coffee, and 2 ice coffees "Add"
can i order 5 coffees "Add"
Please remove the burger from my order "Delete"
Can you swap the burger for a cheese pizza "Update"
Please remove the burger from my order  "Delete"
Add 3 Quesadilla "Add"
Add 3 Quesadilla    "Add"
Can i get a sandwich "Add"
Please cancel my last order which was a burger "burger"
I want a pizza "pizza"


# Reading Intents

In [130]:
pizzaI = pd.read_excel("LabelledTaskMaster/pizzaLabelled.xlsx")
coffeeI = pd.read_excel("LabelledTaskMaster/coffeeLabelled.xlsx")

In [150]:
pizzaI.loc[pizzaI['Intent'] == 'add', 'Intent'] = 'Add'
pizzaI.loc[pizzaI['Intent'] == 'delete', 'Intent'] = 'Delete'
pizzaI.loc[pizzaI['Intent'] == 'modify', 'Intent'] = 'Update'
coffeeI.loc[coffeeI['Intent'] == 'add', 'Intent'] = 'Add'
coffeeI.loc[coffeeI['Intent'] == 'delete', 'Intent'] = 'Delete'
coffeeI.loc[coffeeI['Intent'] == 'modify', 'Intent'] = 'Update'

In [152]:
coffeeI['Intent'].unique()

array(['Add', 'Delete', 'Update'], dtype=object)

In [123]:
readintent = MakeMenu(Database)

In [157]:
readintent.getIntent(pizzaI['Statements'][0])

I want to order a pizza from Bertuccis in Chelmsford, MA


'Add'

In [234]:
def getEval(df):
    ipred = []
    ireal = np.array(df['Intent'])
    readintent = MakeMenu(Database)
    for i in df['Statements']:
        ipred.append(readintent.getIntent(i))
    z = precision_recall_fscore_support(ireal, ipred, average=None, labels=['Add', 'Delete', 'Update'])
    return z, ipred, ireal

In [189]:
a, b, c = getEval(coffeeI)

C:\Users\risha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\risha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [178]:
d, e, f = getEval(pizzaI)

In [239]:
accuracy_score(b, c)

0.8038277511961722

In [237]:
a, b, c = getEval(coffeeI.append(pizzaI))

In [240]:
a

(array([0.86910995, 0.5       , 0.        ]),
 array([0.94318182, 0.09090909, 0.        ]),
 array([0.90463215, 0.15384615, 0.        ]),
 array([176,  22,  11], dtype=int64))

In [238]:
precision_recall_fscore_support(b, c, average='weighted')

C:\Users\risha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.8636907351022184, 0.8038277511961722, 0.8296656734918602, None)

In [214]:
accuracy_score(b, c)

0.8038277511961722

In [245]:
a = MakeMenu(Database)
for i in audioText:
    print(i, a.getIntent(i))

Hi  can I order one burger and fries  Add
I would like to order one large pizza and one soda  Add
Please remove the burger from my order  Delete
I would like to get a coffee  too  Add
Hi  I would like to order one cheeseburger and five fries and three Coke and 7 Sandwich  Add
remove one cup two Delete
